In [42]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"
API_KEY = "carol_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


IPython autoawait is `on`, and set to use `asyncio`


In [43]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. connect to DM

In [44]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


In [45]:
#Paste in invitation from Alice agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '7ed3d9da-83a3-440b-8c6e-64bbbc0c5da2', 'label': 'Alice', 'recipientKeys': ['2NXtAKWpEXaBsjveidkvu4rtjhMV3PWwCctbEj3Zicmu'], 'serviceEndpoint': 'http://192.168.65.3:8020'}

In [46]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection Handler Called
Connection c5f441dd-41d8-4bdb-a041-0a5566e4a450 in State invitation
Connection Handler Called
Connection c5f441dd-41d8-4bdb-a041-0a5566e4a450 in State request
Connection {'their_label': 'Alice', 'their_role': 'inviter', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 10:15:42.020199Z', 'routing_state': 'none', 'state': 'request', 'my_did': '8KjR51tbaUQuYg7knMDfoZ', 'connection_id': 'c5f441dd-41d8-4bdb-a041-0a5566e4a450', 'created_at': '2023-01-03 10:15:41.980086Z', 'accept': 'manual', 'invitation_mode': 'once', 'request_id': 'c1b69a27-ede8-4f19-b4ef-f352d56c9fd7', 'invitation_key': '2NXtAKWpEXaBsjveidkvu4rtjhMV3PWwCctbEj3Zicmu'}
Connection Handler Called
Connection c5f441dd-41d8-4bdb-a041-0a5566e4a450 in State response
Connection Handler Called
Connection c5f441dd-41d8-4bdb-a041-0a5566e4a450 in State active


### switch to DM

In [47]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])
dm_connection = connection["connection_id"]


Alice AGENT CONNECTION
{'their_label': 'Alice', 'their_role': 'inviter', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 10:15:48.506762Z', 'routing_state': 'none', 'state': 'active', 'my_did': '8KjR51tbaUQuYg7knMDfoZ', 'connection_id': 'c5f441dd-41d8-4bdb-a041-0a5566e4a450', 'their_did': 'P28K76EisDuQXgK1BBJc8M', 'created_at': '2023-01-03 10:15:41.980086Z', 'accept': 'manual', 'invitation_mode': 'once', 'request_id': 'c1b69a27-ede8-4f19-b4ef-f352d56c9fd7', 'invitation_key': '2NXtAKWpEXaBsjveidkvu4rtjhMV3PWwCctbEj3Zicmu'}
State: active


In [48]:
# dm_connection = 'e75d05c2-aab0-47d1-b2e5-43415e076121'
print(dm_connection)

c5f441dd-41d8-4bdb-a041-0a5566e4a450


In [65]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'their_label': 'Bob', 'their_role': 'invitee', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 11:26:27.681139Z', 'routing_state': 'none', 'state': 'active', 'my_did': 'HHG2eTSodgTkGHU2QQUDBY', 'connection_id': '88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e', 'their_did': 'GAAHQwGB13skFWsLAgYdRZ', 'created_at': '2023-01-03 11:25:50.175600Z', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq'}]}


## 4. connect to MUSIC

In [50]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection ID 88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '082d1357-64ec-4653-84af-777dd838b48f', 'label': 'Carol', 'serviceEndpoint': 'http://192.168.65.3:8080', 'recipientKeys': ['AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq']}
Connection Handler Called
Connection 88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e in State invitation
Connection Handler Called
Connection 88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e in State request


### switch to MUSIC

In [51]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e in State response
ACCEPT REQUEST
{'their_label': 'Bob', 'their_role': 'invitee', 'rfc23_state': 'response-sent', 'updated_at': '2023-01-03 11:26:26.727559Z', 'routing_state': 'none', 'state': 'response', 'my_did': 'HHG2eTSodgTkGHU2QQUDBY', 'connection_id': '88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e', 'their_did': 'GAAHQwGB13skFWsLAgYdRZ', 'created_at': '2023-01-03 11:25:50.175600Z', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq'}
state response


In [52]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': 'fddba848-a864-4faa-b6b5-99481af42a21'}
Connection Handler Called
Connection 88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e in State active


In [53]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

{'their_label': 'Bob', 'their_role': 'invitee', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 11:26:27.681139Z', 'routing_state': 'none', 'state': 'active', 'my_did': 'HHG2eTSodgTkGHU2QQUDBY', 'connection_id': '88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e', 'their_did': 'GAAHQwGB13skFWsLAgYdRZ', 'created_at': '2023-01-03 11:25:50.175600Z', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq'}
Is Active? active
88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e


In [13]:
# music_connection = 'b616d290-fb08-4a09-9ab4-314fe92f0f25'

In [61]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'their_label': 'Bob', 'their_role': 'invitee', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 11:26:27.681139Z', 'routing_state': 'none', 'state': 'active', 'my_did': 'HHG2eTSodgTkGHU2QQUDBY', 'connection_id': '88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e', 'their_did': 'GAAHQwGB13skFWsLAgYdRZ', 'created_at': '2023-01-03 11:25:50.175600Z', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq'}]}


In [54]:
response = await agent_controller.connections.get_connection(music_connection)
print(response)

{'their_label': 'Bob', 'their_role': 'invitee', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 11:26:27.681139Z', 'routing_state': 'none', 'state': 'active', 'my_did': 'HHG2eTSodgTkGHU2QQUDBY', 'connection_id': '88d9010a-3b97-4319-8ec8-0aa0a6a5ae5e', 'their_did': 'GAAHQwGB13skFWsLAgYdRZ', 'created_at': '2023-01-03 11:25:50.175600Z', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq'}


In [60]:
response = await agent_controller.connections.remove_connection('c5f441dd-41d8-4bdb-a041-0a5566e4a450')

## 9. exchange DID with DM

In [62]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
dm_did = 'None'

my DID X8HWQruWXrv8s6BBLXm53B


In [63]:
def messages_handler(payload):
    global dm_did
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)
    dm_did = payload['content']
    print('DM DID', dm_did)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


In [17]:
# await agent_controller.messaging.trust_ping(dm_connection, "hello, world!")

In [64]:
response = await agent_controller.messaging.send_message(dm_connection, 'sp_'+my_did)
print(response)

ClientResponseError: 404, message='connection record not found: c5f441dd-41d8-4bdb-a041-0a5566e4a450. WalletItemNotFound.', url=URL('http://carol-agent:8081/connections/c5f441dd-41d8-4bdb-a041-0a5566e4a450/send-message')

Error during POST /connections/c5f441dd-41d8-4bdb-a041-0a5566e4a450/send-message: 404, message='connection record not found: c5f441dd-41d8-4bdb-a041-0a5566e4a450. WalletItemNotFound.', url=URL('http://carol-agent:8081/connections/c5f441dd-41d8-4bdb-a041-0a5566e4a450/send-message')


In [39]:
print(dm_did)

PQRXDxdGqQGSZ8z69p4xZP
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': '1a4b3f99-d1c8-4ac3-a9a4-f6af0e5a141b', 'content': 'PQRXDxdGqQGSZ8z69p4xZP', 'state': 'received'} e869f819-eb25-45ec-ac99-0b70bb8aa9a7
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': 'efc5a82d-62f9-4bcf-8e6c-c3624bcfd00b', 'content': 'PQRXDxdGqQGSZ8z69p4xZP', 'state': 'received'} e869f819-eb25-45ec-ac99-0b70bb8aa9a7


## request proof from MUSIC (prove that MUSIC belongs to DM) 

In [22]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)


Subscribing too: present_proof


### switch to MUSIC

In [23]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof from MUSIC")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "author", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "origin", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "owner", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "time", "restrictions": [{"issuer_did": dm_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "owner",
            "restrictions": [{"issuer_did": agent.did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )



#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Ownership of DM",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": music_connection,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

{'result': {'did': 'X8HWQruWXrv8s6BBLXm53B', 'verkey': 'HRL7SCYGvMzoj6QJJ7MymnC3WZ7fXDrpvwV56sPT3zTr', 'posture': 'public'}}
Request proof from MUSIC


In [24]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

Handle present proof
Role verifier, Exchange ce2ff95e-afb0-4b8d-920c-04692e96cc94 in state request_sent
{'role': 'verifier', 'auto_present': False, 'presentation_exchange_id': 'ce2ff95e-afb0-4b8d-920c-04692e96cc94', 'trace': False, 'updated_at': '2023-01-03 09:40:47.416157Z', 'presentation_request': {'name': 'Proof of Ownership of DM', 'version': '1.0', 'requested_attributes': {'0_author_uuid': {'name': 'author', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_origin_uuid': {'name': 'origin', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_owner_uuid': {'name': 'owner', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_time_uuid': {'name': 'time', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {}, 'nonce': '857932762503873741699352'}, 'thread_id': '3264ffcd-fed8-434e-9406-9eacf5ad7d7c', 'state': 'request_sent', 'connection_id': '9756dfc7-8c0e-483c-a61f-3eeda5037f44', 'created_at': '2023-01-03 09:40:47.

### switch to MUSIC

In [25]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

Handle present proof
Role verifier, Exchange ce2ff95e-afb0-4b8d-920c-04692e96cc94 in state verified


In [26]:
print(verify['state'])
music_belongs_to_dm = verify['state'] == 'verified'
print(music_belongs_to_dm)

verified
True


## request new VC from DM

In [27]:
agent_controller.remove_all_listeners("basicmessages")

In [28]:
def request_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle request", payload, connection_id)


request_listener = {
    "handler": request_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([request_listener], defaults=True)

Subscribing too: basicmessages


### switch to DM

In [30]:
response = await agent_controller.messaging.send_message(dm_connection, str(music_belongs_to_dm))
print(response)

{}
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': '82defc0c-75cc-4f06-bfff-6306c3da31f1', 'content': 'revoke: True', 'state': 'received'} e869f819-eb25-45ec-ac99-0b70bb8aa9a7


In [31]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential


Handle Credentials
Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: offer_received
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 09:55:09'}]


In [32]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: offer_received
Being offered: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 09:55:09'}]


In [34]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: request_sent
Handle Credentials
Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: request_sent
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 09:55:09'}]
Handle Credentials
Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: credential_received
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 09:55:09'}]


In [35]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Credentia of Possesion")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: credential_acked
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 09:55:09'}]
Credential exchange 7ec0f73f-e3b9-4e77-b036-94dead6e90f7, role: holder, state: credential_acked
Handle present proof
Role prover, Exchange ec1903e9-995b-4b80-96d0-174f5936d65c in state request_received


## present proof of possesion to MUSIC

In [36]:
response = await agent_controller.proofs.get_records()
print((response['results'][1]))

print('\n')

state = response['results'][1]["state"]
presentation_exchange_id = response['results'][1]['presentation_exchange_id']
presentation_request = response['results'][1]['presentation_request']

print('Presentation Exchange ID')
print(response['results'][1]['presentation_exchange_id'])
print('Presentation Request Object')
print(response['results'][1]['presentation_request'])
print('Requested Attributes')
print(response['results'][1]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][1]['presentation_request']['requested_attributes']
print('Requested Predicates')
print(response['results'][1]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][1]['presentation_request']['requested_predicates']

{'role': 'prover', 'presentation_exchange_id': 'ec1903e9-995b-4b80-96d0-174f5936d65c', 'trace': False, 'updated_at': '2023-01-03 09:55:59.131371Z', 'presentation_request': {'name': 'Proof of Ownership', 'version': '1.0', 'requested_attributes': {'0_possession_uuid': {'name': 'possession', 'restrictions': [{'issuer_did': 'None'}]}, '0_time_uuid': {'name': 'time', 'restrictions': [{'issuer_did': 'None'}]}}, 'requested_predicates': {}, 'nonce': '824512507423231212313172'}, 'thread_id': 'f22aecef-9e4c-4022-b311-ed5c73a67276', 'state': 'request_received', 'connection_id': '9756dfc7-8c0e-483c-a61f-3eeda5037f44', 'created_at': '2023-01-03 09:55:59.131371Z', 'initiator': 'external', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'f22aecef-9e4c-4022-b311-ed5c73a67276', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9m

In [37]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }


for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {}, 'self_attested_attributes': {}}

XXX
{}
{}
{}
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': 'ec82c284-da26-4647-8e30-e9305e155c5d', 'content': 'revoke: True', 'state': 'received'} e869f819-eb25-45ec-ac99-0b70bb8aa9a7
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': 'a8d19b23-84f6-4d20-9e45-38b6434d621b', 'content': 'revoke: True', 'state': 'received'} e869f819-eb25-45ec-ac99-0b70bb8aa9a7
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': '31fcf79c-ac9e-4f86-8b9e-22638b7c41f8', 'content': 'revoke: True', 'state': 'received'} e869f819-eb25-45ec-ac99-0b70bb8aa9a7
Handle request {'connection_id': 'e869f819-eb25-45ec-ac99-0b70bb8aa9a7', 'message_id': '73a56f6d-3c73-4b23-882e-cb516d9707e8', 'content': 'revoke: True

In [106]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'state': 'presentation_sent', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'possession': '14346002616261423285990950419275147645475068884382384349932040365790330023719', 'time': '49877962901940461283957341209906430548424852941388192104851037378149845680243'}, 'a_prime': '28575395726803784489509548117653148574170953588337373772127842094306148101180633028985316504899829041962739903997909541941362934193568776097798135300928609739929742768941889082066411568210903642646217353985292979377500568094160487641011328500401014108204508494300584181167224832850440767217121685510010760505191391048192628762417538223644074599230668837005186710113065435477883988370980596203996532560733277890018283725348826923045260888625535344291842317537364322441459081696273210783861137933512257034975031801604027475293066220939714000420767991030275172481153010935239504205821067835426692074466120668751489701603', 'e': '384979833704859893052160243414379161425962470240725295376

In [66]:
response = await agent_controller.terminate()
print(response)

None
